EDA on subset of data provided by crowdsourcing / lending platform.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Duplicate-values" data-toc-modified-id="Duplicate-values-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Duplicate values</a></span></li><li><span><a href="#Outlier" data-toc-modified-id="Outlier-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Outlier</a></span></li><li><span><a href="#Missing-values" data-toc-modified-id="Missing-values-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Missing values</a></span><ul class="toc-item"><li><span><a href="#Identification-of-missing-values" data-toc-modified-id="Identification-of-missing-values-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Identification of missing values</a></span></li><li><span><a href="#Handling-of-missing-values" data-toc-modified-id="Handling-of-missing-values-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Handling of missing values</a></span></li></ul></li><li><span><a href="#Additional-columns-and-features" data-toc-modified-id="Additional-columns-and-features-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Additional columns and features</a></span></li></ul></li><li><span><a href="#Export-pre-processed-data" data-toc-modified-id="Export-pre-processed-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export pre-processed data</a></span></li></ul></div>

## Data Preparation


In [60]:
# import required libraries

import numpy as np
import pandas as pd 
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

### Import data


In [3]:
# Check structure and separator of the dataset

df_input = pd.read_csv("funding_data.csv",
                       sep="/n",
                       engine='python',
                       nrows=2)
df_input

,# funded_amount# loan_amount# activity# sector# use# country_code# country# region# currency# term_in_months# lender_count# borrower_genders# repayment_interval
0,0#300.0#300.0#Fruits & Vegetables#Food#To buy ...
1,1#575.0#575.0#Rickshaw#Transportation#to repai...


In [4]:
# Import data

df = pd.read_csv("funding_data.csv", sep="#", index_col=0)
df.head()

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
0,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,12.0,12,female,irregular
1,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,11.0,14,"female, female",irregular
2,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,43.0,6,female,bullet
3,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,11.0,8,female,irregular
4,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,14.0,16,female,monthly


In [5]:
# Check df shape

print('The dataframe has ' + str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' columns.')

The dataframe has 671205 rows and 13 columns.


In [6]:
# Check summary of central tendency and dispersion of the numeric columns 

df.describe()

,funded_amount,loan_amount,term_in_months,lender_count
count,671205.000000,671205.000000,671205.000000,671205.000000
mean,785.995061,842.397107,13.739022,20.590922
std,1130.398941,1198.660073,8.598919,28.459551
min,0.000000,25.000000,1.000000,0.000000
25%,250.000000,275.000000,8.000000,7.000000
50%,450.000000,500.000000,13.000000,13.000000
75%,900.000000,1000.000000,14.000000,24.000000
max,100000.000000,100000.000000,158.000000,2986.000000


**To Note**
- large distance between maximum and 75 percentile value in each of the dataframes' columns indicates that some outliers / extreme values are present 
- outliers should be removed from the dataset and / or instead of column's mean that would skew calculations, the median or mode should be chosen

Glance at one of the extreme values:

There is only one loan in the data set that is greater than 50,000 USD. It was granted to an individual female in Haiti by 2986 lenders that invested on average 33.5 USD. The loan has a payment term of 75 month and is for an agricultural investment.

In [7]:
df.loc[df[' loan_amount']>50000,:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
70499,100000.0,100000.0,Agriculture,Agriculture,create more than 300 jobs for women and farmer...,HT,Haiti,Les Cayes,USD,75.0,2986,female,irregular


In [8]:
# Check summary statistics of categorical columns 

df.describe(include=['O']) 

,activity,sector,use,country_code,country,region,currency,borrower_genders,repayment_interval
count,671205,671205,666973,671197,671205,614405,671205,666984,671205
unique,163,15,424912,86,87,12695,67,11298,4
top,Farming,Agriculture,to buy a water filter to provide safe drinking...,PH,Philippines,Kaduna,PHP,female,monthly
freq,72955,180302,5217,160441,160441,10000,160440,426502,342717


**To Note**: 

- surprisingly large number of unique values in the column borrower_genders
- 87 unique countries present in dataset but only 86 unique country codes; indicates presence of missing values.

In [9]:
# Check spelling of the column names

df.columns

Index([' funded_amount', ' loan_amount', ' activity', ' sector', ' use',
       ' country_code', ' country', ' region', ' currency', ' term_in_months',
       ' lender_count', ' borrower_genders', ' repayment_interval'],
      dtype='object')

The leading spaces present in the column names will be removed so that the data can queried more easily.

In [10]:
# Remove leading spaces in column names

df.columns = df.columns.str.lstrip()

In [11]:
# Check correlation of numeric values

df.corr(numeric_only=True)

,funded_amount,loan_amount,term_in_months,lender_count
funded_amount,1.000000,0.945044,0.149310,0.849168
loan_amount,0.945044,1.000000,0.184795,0.798697
term_in_months,0.149310,0.184795,1.000000,0.227283
lender_count,0.849168,0.798697,0.227283,1.000000


**To note**

- strong positive correlation (approx. 94.5%) between the funded amount and the loan amount
--> indicates that platforms's business model - being the intermediate party that connects lenders and borrowers - works
--> correlation coefficient of approx. 0.95 displays a strong linear relationship between the value that borrowers ask from the lenders and what they actually receive as funding


- quite strong correlation between the funded amonunt and the number of lenders (lender_count) 
--> indicates that for higher funded amounts, more people invest 
--> indicates that to fund high profile / bigger funding sums, more investors are needed as they rarely receive significant higher donaction amounts than smaller projects


- terms of repayment in an investment are correlated only very weakly with other variables 
--> indicates that investors are not as focused on the speed of receiving back their invested amount as the crowdfunding website calls upon the investors purpose behind participating in such investments


**Summary of first impressions**

The raw dataset consists of 671,205 rows and 13 columns. The strong correlation between the column funded_amount and the column loan_amount is a first indication that the company operates effectively as the intermediate between lender and borrower. Moreover, there seem to be some missing values and some outliers present in the dataset.

### Duplicate values

Duplicate records (duplicate rows) are defined as identical rows in the dataframe (df). This means, that a pair of duplicate records is present when each value in each of their column coincide. 


In [12]:
# count of duplicate records

print('There are ' + (df.duplicated().sum()).astype(str) + ' duplicate values in the dataset.')

There are 24372 duplicate values in the dataset.


The identified duplicate rows coincide in each value which means that they match the above stated definition of duplicated records. Thus, the duplicated rows will be deleted in the following. Thereby, only the first record (per duplicate occurence) will be kept in the dataset. In addition, the index will be reset.

In [13]:
# Drop duplicated records

df.drop_duplicates(keep='first',inplace=True)

In [14]:
# Check if duplicated rows were dropped

df.loc[df.duplicated(keep=False) == True, :]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval


In [15]:
# reset index and drop former index

df.reset_index(inplace=True,drop=True)

The dataset is now free of duplicates.

### Outlier 

Generally, outlier are defined as data points that lie abnormally far from other values. As there is one investment in the dataset that has an abnormally large loan and funded amount, this data point was considered as a potential outlier.
This data point is displayed below. 

However, this entry will not be considered as an outlier and thus, will not be removed from the dataset. The reason for this is that information would get lost if it was deleted. It is assumed that the company's revenue is a derivative of both the number of investments as well as their individual amount. This means that the company's profits from a large number of investments on their platform and also from investments that are large, assuming that their profit is a commission of the funded amount. 

In this case the particular investment was in the sector agriculture and included an investment to create jobs for over 300 people. Potentially, the company could therefore find new expansion opportunities for their current offering of loans.

In [16]:
# check outlier

df.loc[df['loan_amount']>50000,:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
69745,100000.0,100000.0,Agriculture,Agriculture,create more than 300 jobs for women and farmer...,HT,Haiti,Les Cayes,USD,75.0,2986,female,irregular


### Missing values

#### Identification of missing values


In this analysis, missing values are defined as
1. Synonyms or standardized substitute values (e.g., -9999 or '?') or
2. null (np.nan) values



Below, the identification of missing values will be performed based on the exploration of the column data types as well as based on the count of np.nan values in each column. 
Analysing the data types can help identify inconsistencies in the format on hand vs. the column content. Thereby, a mismatch can be an indication that synonyms were used or that a prior imputation of the respective values has happened. Moreover, the identification and distribution of present np.nan values will be attempted to analyse the extent of null values in the dataset which is required to determine the further handling of these values.

In [17]:
# Check data type of each columns
                                                                              
df.dtypes   

funded_amount         float64
loan_amount           float64
activity               object
sector                 object
use                    object
country_code           object
country                object
region                 object
currency               object
term_in_months        float64
lender_count            int64
borrower_genders       object
repayment_interval     object
dtype: object

**To note**

There seems to be no inconsistency between the expected and present data types.

In [18]:
# Check non-null values 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646833 entries, 0 to 646832
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   funded_amount       646833 non-null  float64
 1   loan_amount         646833 non-null  float64
 2   activity            646833 non-null  object 
 3   sector              646833 non-null  object 
 4   use                 642934 non-null  object 
 5   country_code        646825 non-null  object 
 6   country             646833 non-null  object 
 7   region              590670 non-null  object 
 8   currency            646833 non-null  object 
 9   term_in_months      646833 non-null  float64
 10  lender_count        646833 non-null  int64  
 11  borrower_genders    642945 non-null  object 
 12  repayment_interval  646833 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 64.2+ MB


In [19]:
# Check null values in % per column

round(df.isnull().sum() / len(df) * 100, 2)

funded_amount         0.00
loan_amount           0.00
activity              0.00
sector                0.00
use                   0.60
country_code          0.00
country               0.00
region                8.68
currency              0.00
term_in_months        0.00
lender_count          0.00
borrower_genders      0.60
repayment_interval    0.00
dtype: float64

In [20]:
# Check if null values in the columns use, borrower_genders, and region all stem from the same rows of data

df.loc[(df['use'].isnull()) & (df['borrower_genders'].isnull()) & (df['region'].isnull()),:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
140,2975.0,2975.0,Food Production/Sales,Food,NaN,TZ,Tanzania,NaN,TZS,10.0,110,NaN,monthly
145,1200.0,1200.0,Personal Expenses,Personal Use,NaN,PE,Peru,NaN,PEN,20.0,44,NaN,monthly
170,4250.0,4250.0,Catering,Food,NaN,TZ,Tanzania,NaN,TZS,10.0,116,NaN,monthly
412,2350.0,2350.0,Beauty Salon,Services,NaN,TZ,Tanzania,NaN,TZS,10.0,75,NaN,monthly
414,725.0,725.0,Agriculture,Agriculture,NaN,SV,El Salvador,NaN,USD,20.0,19,NaN,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636081,5625.0,10000.0,Weaving,Arts,NaN,BT,Bhutan,NaN,USD,14.0,210,NaN,irregular
637157,1975.0,1975.0,Home Energy,Personal Use,NaN,PS,Palestine,NaN,USD,27.0,39,NaN,monthly
637995,800.0,1600.0,Furniture Making,Manufacturing,NaN,HT,Haiti,NaN,HTG,13.0,27,NaN,irregular
646812,0.0,25.0,Livestock,Agriculture,NaN,KE,Kenya,NaN,KES,13.0,0,NaN,monthly


In [21]:
# Check if null values in the columns use, borrower_genders, and region exists in many or in just a few countries 

df.loc[(df['use'].isnull()) & (df['borrower_genders'].isnull()) & (df['region'].isnull()),'country'].value_counts()

Colombia         769
Kenya            558
El Salvador      533
United States    159
Uganda           151
                ... 
South Africa       1
Kyrgyzstan         1
Chile              1
Georgia            1
Bhutan             1
Name: country, Length: 69, dtype: int64

**To note** 

- null values appear in a wide range of countries. 

Next, the rows with null values will be further looked at. Rows with null values in multiple columns could be considered for deletion if they lack the required information content and/or potential to fill them. Null values in multiple columns could also indicate a connection between the columns such as one missing value increasing the chance / causing the null value of another column.

However, deleting rows with only one or very limited numbers of null values in their columns would result in a loss and potential skew of data contained in the dataset. Thus, it will be checked if null values in one column systematically appear with null values in one or more columns.

In [22]:
# Check if there are null values in the column 'use' but not in the column 'borrower_genders'

df.loc[(df['use'].isnull()) & (~df['borrower_genders'].isnull()),:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
76238,200.0,5000.0,Technology,Services,NaN,US,United States,NaN,USD,24.0,2,female,bullet
115857,2475.0,2475.0,Food Production/Sales,Food,NaN,RW,Rwanda,NaN,RWF,4.0,21,"male, female, female, female, female, female, ...",irregular
183284,235.0,4000.0,Technology,Services,NaN,US,United States,NaN,USD,24.0,9,female,bullet
191009,650.0,650.0,Agriculture,Agriculture,NaN,SV,El Salvador,Osicala,USD,15.0,21,female,bullet
204015,100.0,5000.0,Clothing,Clothing,NaN,US,United States,NaN,USD,12.0,2,male,bullet
206788,0.0,5000.0,Food Production/Sales,Food,NaN,US,United States,NaN,USD,24.0,0,female,bullet
243095,0.0,3000.0,Services,Services,NaN,US,United States,NaN,USD,18.0,0,female,bullet
287353,750.0,1175.0,Clothing Sales,Clothing,NaN,KE,Kenya,Maua,KES,14.0,11,male,monthly
354828,100.0,100.0,Farming,Agriculture,NaN,MZ,Mozambique,Boane,MZN,7.0,4,male,monthly
441552,200.0,200.0,Home Appliances,Personal Use,NaN,KH,Cambodia,Kampong Speu,KHR,8.0,7,"female, male, female, female, male",monthly


Rows exist with null values in the column 'use' but not in the column 'borrower_genders'.

In [23]:
# Check if there are null values in the column 'use' but not in the column 'borrower_genders'

df.loc[(df['borrower_genders'].isnull()) & (~df['use'].isnull())]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval


There are no rows with null values in the column 'use' but not in the column 'borrower_genders'.

In [24]:
# Check if there are null values in the column 'use' but not in the column 'region'

df.loc[(df['use'].isnull()) & (~df['region'].isnull()),:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval
191009,650.0,650.0,Agriculture,Agriculture,NaN,SV,El Salvador,Osicala,USD,15.0,21,female,bullet
287353,750.0,1175.0,Clothing Sales,Clothing,NaN,KE,Kenya,Maua,KES,14.0,11,male,monthly
354828,100.0,100.0,Farming,Agriculture,NaN,MZ,Mozambique,Boane,MZN,7.0,4,male,monthly
441552,200.0,200.0,Home Appliances,Personal Use,NaN,KH,Cambodia,Kampong Speu,KHR,8.0,7,"female, male, female, female, male",monthly
467261,700.0,700.0,Pigs,Agriculture,NaN,KH,Cambodia,"Battambang province, Sangke district",KHR,11.0,16,"female, female",monthly


There are rows with null values in the column 'use' but not in the column 'region'.

In [25]:
# Check if there are null values in the column 'use' but not in the column 'region'

df.loc[(df['borrower_genders'].isnull()) & (~df['region'].isnull()),:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval


There are no rows with null values in the column 'use' but not in the column 'region'.

**To note** 

- the columns 'use', 'country_code', 'region', 'borrower_genders' have null values. 
- the highest percentage of null values is present in the column 'region' (8.7%) 
- other columns with null values are the columns 'use' (0.6%), 'borrower_genders' (0.6%)., and 'country code' (0.0%).

Thereby, a null value in the column 'borrower_genders' is always accompanied by a null value in the column 'region' and 'use'. This, however, is not the case vice versa.

Both the 'region' and the 'use' column have missing values in entries where the other columns have no missing values.

#### Handling of missing values


Next, one of the following approaches will be considered for missing values:
1. For numeric values: fill missing values with median value
2. For categorical values: fill missing values with mode value
3. Deletion of entire column or row if not needed for analysis



In [26]:
# Deep dive into the null values in the country_code country

df.loc[df['country_code'].isnull(),'country'].unique()

array(['Namibia'], dtype=object)

Namibia is the only country that lacks a value in the column 'country_code'.

In [27]:
# Check if Namibia's official country code is present in dataset

df.loc[df['country_code']=='NA',:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval


Namibia's country code ('NA') is not yet present in the dataset. As the official country iso code is publicly available and as country iso codes are standardized combination of characters, Namibia's two digit country code will be filled with 'NA'.  The format of the alpha-code2 was chosen to align with the existing structure of the other country codes. 

Assumption: In this case the missing values in the country_code column might stem from the fact that Namibia's country code 'NA' was, when extracted and transformed into a csv, interpreted by the system as a 'not applicable (NA)' value. Should this issue persist in the future it would be advisable to account for it in future data extractions.

In [28]:
# replace missing values in country code column

df.loc[:, "country_code"].replace(np.nan, 'NA', inplace=True)

In [29]:
# Check that there are no more null values

df.loc[df['country_code'].isnull(),:]

,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,term_in_months,lender_count,borrower_genders,repayment_interval


Null values in the column 'region':

In [30]:
# Number of missing values in column region

print('There are', df['region'].isnull().sum(), 'missing values present in the column region.')

There are 56163 missing values present in the column region.


In [31]:
# see if null values in region column exist only in a few countries

df.loc[df['region'].isnull(),'country'].value_counts()

El Salvador      20222
Kenya             8547
Rwanda            6080
United States     5172
Senegal           3229
                 ...  
China                1
Georgia              1
Chile                1
Guam                 1
Bhutan               1
Name: country, Length: 72, dtype: int64

Null values in region column exist in many countries. 

In [32]:
# example of values for region in El Salvador

df.loc[df['country']=='El Salvador','region'].value_counts()

San Miguel                                   2737
Gotera                                       2300
Usulután                                     2230
Ciudad El Triunfo                            2176
El Transito                                  1842
Osicala                                      1712
La Unión                                     1615
Sensuntepeque                                1194
Anamoros                                     1074
Ciudad Barrios                                982
Jiquilisco                                    707
San Vicente                                   675
San Miguel, San Miguel                         68
San Rafael de Oriente, San Miguel              35
San Francisco Gotera, Morazan                  30
Corinto, Morazan                               29
Zacatecoluca                                   26
Jiquilisco, Usulutan                           22
El Transito, San Miguel                        18
San Carlos, Morazan                            18


**To note**

Missing values in column 'region'

- with approx. 8.7% of null values, it has the highest percentage of missing values in the dataset
- missing values are quite spead out; are present in 72 countries 
- there seems to be a lack of standardization of the column: different granularity of data (see example above: 'region' == 'San Miguel' , 'region' == 'San Miguel, San Miguel', 'region' == 'San Jorge, San Miguel').



**Handling of missing values in the column 'region'**

For the following (exploratory) data analysis the region information will not be utilized. The column 'region' contains information on the different geographical regions within a country; however, for this analysis, the granularity on a country level is sufficient for the planned analysis. Hence, the column will be dropped in the following. If required, future analysis can then increase the level (geographical) granularity and include the column 'region'. 

In [33]:
# drop column 'region'

df.drop(columns='region', inplace=True)

Null values in the column 'use':

In [34]:
# number of missing values in the column 'use'

print('There are', df['use'].isnull().sum(),
      'missing values present in the column use.')

There are 3899 missing values present in the column use.


In [35]:
# see if missing values are concentrated for entries in specific countries

df.loc[df['use'].isnull(),'country'].value_counts()

Colombia         769
Kenya            559
El Salvador      534
United States    164
Uganda           151
                ... 
Chile              1
Kyrgyzstan         1
Puerto Rico        1
China              1
Bhutan             1
Name: country, Length: 69, dtype: int64

Null values in the column 'use' exist in many countries. 

**To note**

Missing values in column 'use'

- 3899 null values 
- contains short descriptions of the borrower's planned purchase for which they require an investment

**Handling of missing values in the column 'use'**


Similar to the column 'region', the column 'use' will be not be utilized in the course of this analysis as it contains too detailed information for the planned analysis.

In [36]:
# drop the column 'use'

df.drop(columns='use', inplace=True)

Null values in the column 'borrower_genders':


In [37]:
print('There are', df['borrower_genders'].isnull().sum(),
      'missing values present in the column borrower_genders.')

There are 3888 missing values present in the column borrower_genders.


**To note**

Missing values in column 'borrower_genders'
- 3888 missing values

**Handling of missing values in the column 'use'**

The column 'borrower_genders' will be used in the following analysis: thus, its missing values will be filled with the mode. The mode is chosen as the column's scale level is nominal. However, not the overall mode of the column will be used to fill the null values as this bears the risk of oversimplifying the imputation which could lead to skewed data. Instead the mode by country code and sector will be calculated and used to replace the missing values. 

In [38]:
# Check example of occurences of column borrower_genders

df.loc[(df['country_code']=='AL') &  (df['sector']=='Agriculture'),'borrower_genders'].value_counts()

female    361
male      354
Name: borrower_genders, dtype: int64

In [39]:
# Check most common value of example

df.loc[(df['country_code']=='AL') &  (df['sector']=='Agriculture'),'borrower_genders'].value_counts().index.tolist()[0]

'female'

In [40]:
# function to define mode

def find_mode(group_cols=[], mode_col=[]):
    return df.groupby(group_cols,
                      as_index=False).agg(mode=(mode_col, pd.Series.mode))

In [41]:
# create df containing mode for column borrower_genders based on country code and sector

df_mode_gender = find_mode(group_cols=['country_code','sector'],mode_col='borrower_genders')
df_mode_gender

,country_code,sector,mode
0,AF,Arts,female
1,AL,Agriculture,female
2,AL,Arts,female
3,AL,Clothing,female
4,AL,Construction,male
...,...,...,...
985,ZW,Personal Use,"female, female, female, female"
986,ZW,Retail,female
987,ZW,Services,female
988,ZW,Transportation,"[female, male]"


In [42]:
# Check if mode was extracted correctly based on example above

df_mode_gender.loc[(df_mode_gender['country_code'] == 'AL') &
                   (df_mode_gender['sector'] == 'Agriculture'), 'mode']

1    female
Name: mode, dtype: object

In [43]:
# create list with index of all rows where null value in column borrower_genders

missing_gender_list = df.loc[df['borrower_genders'].isna()].index
missing_gender_list

Int64Index([   140,    145,    170,    412,    414,    464,    466,    545,
               594,    614,
            ...
            624480, 624832, 625286, 635308, 635871, 636081, 637157, 637995,
            646812, 646826],
           dtype='int64', length=3888)

In [44]:
for value in missing_gender_list:

    # identify values
    ccode_at_value = df.loc[value, 'country_code']
    sector_at_value = df.loc[value, 'sector']

    # find mode
    mode_gender = df_mode_gender.loc[
        (df_mode_gender['sector'] == sector_at_value) &
        (df_mode_gender['country_code'] == ccode_at_value), 'mode'].values

    # assign values
    df.loc[value, 'borrower_genders'] = mode_gender

In [45]:
# transform array values of previous mode imputation into string values

df['borrower_genders'] = df['borrower_genders'].apply(lambda x: str(x[0]) if isinstance(x,np.ndarray) else x)

**Summary of missing data**

The dataset is now free of missing values. The missing values in the column 'borrower_genders' and 'country_code' were filled and the columns 'use' and 'region' were dropped as they do not contain information that will be used for any analysis planned at this moment in time.

### Additional columns and features

Additional columns: To ease the future analysis, additional columns will be created. The columns will be based on existing columns and aim to extend and ease the analysis of the provided dataset.

Features: To increase the potential information gain from the analysis, the values of the existing columns will be combined to gain new insights.

**New column: 'borrower_type'**

Based on the number and gender of borrowers a new column is created.

In [46]:
# function to define the type of borrower

def gender_categories(sample):
    sample_result = sample.split(",")
    num_male = 0
    num_female = 0

    for sample_result in sample_result:
        cleaned_sample = sample_result.strip()
        if cleaned_sample == 'male':
            num_male += 1
        if cleaned_sample == 'female':
            num_female += 1

    cat = ''

    # Individual borrowers
    if num_male == 0 and num_female == 1:
        cat = 'Individual Female'
    elif num_male == 1 and num_female == 0:
        cat = 'Individual Male'

    # small homogenous group borrowers
    elif num_male == 0 and num_female <= 5:
        cat = 'Small female group'
    elif num_male <= 5 and num_female == 0:
        cat = 'Small male group'

    # large homogenous group borrowers
    elif num_male == 0 and num_female > 5:
        cat = 'Large female group'
    elif num_male > 5 and num_female == 0:
        cat = 'Large male group'

    # Small heterogenous group borrower
    elif (num_male + num_female) <= 5:
        cat = 'Small mixed group'
    # large heterogenous group borrowers
    elif (num_male + num_female) > 5:
        cat = 'Large mixed group'

    else:
        cat = 'female'
    return cat

In [47]:
# create new column with pre-defined categories

df['borrower_type'] = df['borrower_genders'].apply(lambda x: gender_categories(x))
df[['borrower_genders', 'borrower_type']].tail()

,borrower_genders,borrower_type
646828,female,Individual Female
646829,female,Individual Female
646830,"female, female",Small female group
646831,"female, female",Small female group
646832,female,Individual Female


In [48]:
# distribution of borrower types, see which type exists the most

(round(df['borrower_type'].value_counts(normalize=True),2)*100)

Individual Female     65.0
Individual Male       20.0
Small female group     5.0
Large mixed group      4.0
Large female group     3.0
Small mixed group      2.0
Small male group       0.0
Large male group       0.0
Name: borrower_type, dtype: float64

**New column: 'borrower_type_general'**

Based on the gender of borrowers a new column is created.

In [49]:
def borrower_type_change(input_string):

    # lower input case as it will make it
    # easy to check and return value
    input_string = input_string.lower()
    if 'female' in input_string:
        return 'female'
    elif 'male' in input_string:
        return 'male'
    else:
        return 'female and male'
    
df['borrower_type_general'] = df['borrower_type'].apply(lambda x: borrower_type_change(x))

**New Feature: %funded**

The funded amount divided by the loan amount results in the percentage funded. This value can be an indication of how well the company is doing in connecting lenders and borrowers successfully. As the firm earns a provision for each placed investment request, a high level of fully funded projects can increase their chances to be picked by potential borrowers. 

As lenders of the company are defined as purpose driven people, a high level of overall % funded can also be an indication that the lenders support the investment requests and their underlying investment goal.


In [50]:
# funtion to calculate percentage

def percentage_calc(numerator,denominator):
    try:
        return round((numerator / denominator),1)     
    except ZeroDivisionError:
        return 0

In [51]:
# create new column 

df['%funded'] = percentage_calc(df['funded_amount'] , df['loan_amount'])
df.tail(1)

,funded_amount,loan_amount,activity,sector,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,borrower_type,borrower_type_general,%funded
646832,0.0,250.0,Livestock,Agriculture,GH,Ghana,GHS,13.0,0,female,monthly,Individual Female,female,0.0


In [52]:
df['%funded'].value_counts()

1.0    599196
0.6      6237
0.4      5833
0.5      5485
0.7      5173
0.8      4957
0.2      4831
0.0      4642
0.3      4345
0.1      3500
0.9      2632
1.1         2
Name: %funded, dtype: int64

In [53]:
# Check what entries that are 110% funded in the dataset

df.loc[df['%funded']>1,:]

,funded_amount,loan_amount,activity,sector,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,borrower_type,borrower_type_general,%funded
272125,425.0,400.0,General Store,Retail,MZ,Mozambique,MZN,17.0,11,male,monthly,Individual Male,male,1.1
331239,3400.0,3000.0,Farm Supplies,Agriculture,AM,Armenia,USD,38.0,84,male,monthly,Individual Male,male,1.1


There should be some follow-up discussions on how it can happen that a project is overfunded. It could be a system or a human error when entering the data. Potentially, funded amounts that exceed the loan amount could also be acceptable.

**New column: funding_status**

Based on the achieved funding rate, the loans are put into a funding status category.

In [54]:
# funding categories
funding_cat = [
    'not funded', 'up to 20% funded', '>20% funded', '>40% funded',
    '>60% funded', '>80% funded', 'fully funded', 'overfunded'
]

# funding ranges
funding_range = [0 - .001, 0, 0.2, 0.4, 0.6, 0.8, 1 - .001, 1, 100]

# new column funding status
df["funding_status"] = pd.cut(df.loc[:, "%funded"],
                              bins=funding_range,
                              labels=funding_cat)

**New column: invested_per_lender**

The column invested_per_lender reflects the average investment per lender. This can show if investments with a comparably large loan (and funding) achieve the high amounts by a few or by many investors which gives an indication on the typical lender on the platform. However, as there is no additional information available on the lenders on the platform, the average only gives an indication.

In [55]:
df['invested_per_lender'] = percentage_calc(df['funded_amount'] , df['lender_count'])
df['invested_per_lender'] = df['invested_per_lender'].replace(np.nan,0.0)
df.head(1)

,funded_amount,loan_amount,activity,sector,country_code,country,currency,term_in_months,lender_count,borrower_genders,repayment_interval,borrower_type,borrower_type_general,%funded,funding_status,invested_per_lender
0,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,PKR,12.0,12,female,irregular,Individual Female,female,1.0,fully funded,25.0


In [56]:
# distribution of the average investment amount

df['invested_per_lender'].describe()

count    646833.000000
mean         61.646820
std         181.480392
min           0.000000
25%          26.800000
50%          31.600000
75%          43.300000
max        9475.000000
Name: invested_per_lender, dtype: float64

75% of the average investments are 43.3 USD or below. This goes in line with the compnay's defined target lenders: people with a purpose as well as with their crowd investing business model where many people invest (potentially smaller amounts) to fund an investment instead of a few investors funding entire projects.

**New column: continent**

In [57]:
df.country_code.value_counts()

PH    154323
KE     73554
SV     39845
KH     27067
PK     26475
       ...  
AF         2
BT         2
GU         1
MR         1
CI         1
Name: country_code, Length: 87, dtype: int64

In [61]:
# add continent based on country code

def get_continent(col):
    try:
        cn_continent = country_alpha2_to_continent_code(col)
    except:
        cn_continent = 'Unknown' 
    return cn_continent

In [62]:
df['continent_code'] = df.apply(lambda x: get_continent(x['country_code']), axis=1)

In [65]:
# Unknown
df.loc[df['continent_code']=="Unknown", "country"].value_counts()

Timor-Leste    2682
Name: country, dtype: int64

In [66]:
# replace unknown

df.loc[df['continent_code']=="Unknown", "continent_code"] = "AS"

In [67]:
# add full names

mapper = {'AS':'Asia',
          'AF':'Africa',
          'NA':'North America',
          'SA':'South America',
          'OC':'Oceania',
          'EU':'Europe'
         }

df.loc[:,'continent'] = df.loc[:,"continent_code"].map(mapper)

In [68]:
del mapper

In [70]:
# check
df.continent.value_counts()

Asia             306826
Africa           165474
North America     83075
South America     78851
Oceania            7943
Europe             4664
Name: continent, dtype: int64

## Export pre-processed data

The data cleaning is now finished and the cleaned data set is ready to be exported.

In [71]:
df.to_pickle('df_crowdsourcing_after_preprocessing.pkl')